<a href="https://colab.research.google.com/github/fdq09eca/Study/blob/FoTan/thanh_vnmap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# !pip install folium geopandas imgkit
# import folium
# import pandas
# from google.colab import drive
# drive.mount('/content/drive/')

In [381]:
df_rain = pd.read_csv('/content/drive/My Drive/Thanh_map/precipitation.csv')
df_rain = pd.concat([df_rain,df_rain['district'].str.split(',',expand=True)],axis='columns').drop(columns='district')
df_rain = df_rain.rename(columns={0:'District',1:'Province','province':'p_id'})
m_cols= ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct',
       'nov', 'dec']
df_rain['yr_rain'] = df_rain[m_cols].sum(axis='columns')
df_rain.drop(columns=m_cols, inplace = True)
df_rain['District'] = df_rain['District'].str.strip().str.replace('Disctrict','Quan').str.replace('District','Quan')

df_rain.head()

,p_id,District,Province,yr_rain
0,1,Ba Dinh,Ha Noi,0.0
1,1,Hoan Kiem,Ha Noi,0.0
2,1,Tay Ho,Ha Noi,0.0
3,1,Long Bien,Ha Noi,0.0
4,1,Cau Giay,Ha Noi,0.0


In [384]:
url = 'https://data.opendevelopmentmekong.net/dataset/6f054351-bf2c-422e-8deb-0a511d63a315/resource/c906af7a-e7a0-4776-95d4-5ee815dba760/download/District.geojson'
geo_df = geopandas.read_file(url)
geo_df.head()

,OBJECTID,f_code,Province,District,Pop_2009,Code_re,geometry
0,1,AD02,Ca Mau,Nam Can,66229.0,96971,"POLYGON ((105.05492 8.85322, 105.05819 8.85155..."
1,2,AD02,Ca Mau,Dam Doi,182028.0,96970,"POLYGON ((105.16041 9.09551, 105.16228 9.09309..."
2,3,AD02,Ca Mau,Cai Nuoc,137396.0,96969,"POLYGON ((105.08692 9.13987, 105.08773 9.13954..."
3,4,AD02,Bac Lieu,Dong Hai,143590.0,95960,"POLYGON ((105.55273 9.23437, 105.56841 9.17901..."
4,5,AD02,Ca Mau,TP. Ca Mau,216196.0,96964,"POLYGON ((105.17230 9.26629, 105.17518 9.26575..."


In [386]:
df = pd.merge(geo_df, df_rain, how='right', on=['District'])
print(f'''massing rate: {df[df['Province_x'].isnull()].Province_y.value_counts().sum()/df.shape[0]:.2f}''')
df.head()

# df[df['Province_x'].isnull()]['Province_y'].value_counts().index.tolist()
# rename_list = ['TP. Ho Chi Minh', 'Dak Lak']
# dict(df[df['Province_x'].isnull()]['Province_y'].value_counts().index.tolist(),)

massing rate: 0.01


,OBJECTID,f_code,Province_x,District,Pop_2009,Code_re,geometry,p_id,Province_y,yr_rain
0,1.0,AD02,Ca Mau,Nam Can,66229.0,96971,"POLYGON ((105.05492 8.85322, 105.05819 8.85155...",96,Ca Mau,0.000000
1,2.0,AD02,Ca Mau,Dam Doi,182028.0,96970,"POLYGON ((105.16041 9.09551, 105.16228 9.09309...",96,Ca Mau,0.022320
2,3.0,AD02,Ca Mau,Cai Nuoc,137396.0,96969,"POLYGON ((105.08692 9.13987, 105.08773 9.13954...",96,Ca Mau,0.022957
3,4.0,AD02,Bac Lieu,Dong Hai,143590.0,95960,"POLYGON ((105.55273 9.23437, 105.56841 9.17901...",95,Bac Lieu,0.023468
4,7.0,AD02,Bac Lieu,Gia Rai,137249.0,95959,"POLYGON ((105.40669 9.34557, 105.40888 9.34439...",95,Bac Lieu,0.000000


In [0]:
# confirming the saving directory
# %cd '/content/drive/My Drive/Thanh_map' 
# df = df_rain
df['yr_rain'] = df['yr_rain']/df['yr_rain'].max() # scale the rain_level to [0,1].
bins = list(df['yr_rain'].quantile([0, 0.25, 0.5, 0.75, 1])) # make the pattern more obvious.

map_g = folium.Map(location=[14,108],zoom_start=5 , prefer_canvas=True)

folium.Choropleth(geo_data=url,
    name='choropleth',
    data=df,
    columns=['District', 'yr_rain'],
    key_on='feature.properties.District',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,  bins=bins,
    reset=True,
    legend_name='Vitnam Precipitation, 1947-1948').add_to(map_g)

folium.LayerControl().add_to(map_g)

map_g.save('map_precipitation.html')
df.to_csv('vn_2009_geojson.csv')

In [393]:
df_rain['yr_rain'].value_counts(bins=4)

(-0.001043, 0.0107]    353
(0.0107, 0.0215]       212
(0.0215, 0.0322]       124
(0.0322, 0.043]          4
Name: yr_rain, dtype: int64

In [394]:
bins

[0.0, 0.0, 0.4006894766110811, 0.49464192634638016, 1.0]